In this proof of concept we will read & write JSON files in Jupyter notebook. 

1. display the data in the sidecar []
2. edit this data []
3. check that the sidecar will write valid JSON files. 

In [16]:
#import json module to be able to read & write json files 
import json

The first part will include displaying the data in the sidecar by reading the JSON files 

In [14]:
#open the json file in your directory 

file = open('/Users/bjaber/Desktop/package_project/data/sub-1832999514/ses-PNC1/func/sub-1832999514_ses-PNC1_task-rest_acq-singleband_task-rest_bold.json')

In [15]:
#use json.load command to read the file. 

json_data = json.load(file)

In [25]:
#Uncomment below to view data
json_data

{'AcquisitionDateTime': '2010-02-18T19:45:55.120000',
 'AcquisitionMatrixPE': 64,
 'AcquisitionNumber': 1,
 'AcquisitionTime': '19:45:55.120000',
 'BandwidthPerPixelPhaseEncode': 28.409,
 'BaseResolution': 64,
 'ConversionSoftware': 'dcm2niix',
 'ConversionSoftwareVersion': 'v1.0.20190410  (JP2:OpenJPEG) GCC4.8.4',
 'DeidentificationMethod': 'Penn_BSC_profile_v1.0',
 'DerivedVendorReportedEchoSpacing': 0.000550002,
 'DeviceSerialNumber': '35069',
 'EchoTime': 0.032,
 'EffectiveEchoSpacing': 0.000550002,
 'FlipAngle': 90,
 'ImageOrientationPatientDICOM': [1, 0, 0, 0, 1, 0],
 'ImageType': ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC'],
 'ImagingFrequency': 123.259,
 'InPlanePhaseEncodingDirectionDICOM': 'COL',
 'InstitutionAddress': 'Spruce_Street_3400_Philadelphia_20DBC9_Pennsylvania_US_19104',
 'InstitutionName': 'HUP6',
 'MRAcquisitionType': '2D',
 'MagneticFieldStrength': 3,
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'TrioTim',
 'Modality': 'MR',
 'PartialFourier': 1,
 'Pat

The following part is used to edit JSON file data. 

In order to do so, call to the JSON object that was created using the json.load commeand, in this case json_data, and refer to the value that you want to change and edit it. 

In [18]:
# Here we change the value for AcquisionNumber from 1 to 2. 
json_data["AcquisitionNumber"] = 2

In [21]:
#Uncomment below to view edited data
#json_data

In [22]:
#Reverting back to original data
json_data["AcquisitionNumber"] = 1

Checking that the sidecare will write valid JSON files 

In [24]:
def validateJSON(json_data): 
    try: 
        json.loads(json_data)
    except ValueError as err: 
        return False 
    return True 

In [ ]:
validJsonData = 

json.dumps = writes valid json no matter what because it will turn it into a string 